In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("nyc-taxi-inspect-bronze")
    .master("local[*]")
    .config("spark.driver.memory", "6g")
    .config("spark.sql.shuffle.partitions", "16")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/04 13:41:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
bronze_path = "../data/processed/bronze/nyc_taxi/yellow_tripdata_2016-01"
bronze_df = spark.read.parquet(bronze_path)
bronze_df

DataFrame[VendorID: string, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: string, trip_distance: string, pickup_longitude: string, pickup_latitude: string, RatecodeID: string, store_and_fwd_flag: string, dropoff_longitude: string, dropoff_latitude: string, payment_type: string, fare_amount: string, extra: string, mta_tax: string, tip_amount: string, tolls_amount: string, improvement_surcharge: string, total_amount: string, _source_file: string]

In [6]:
bronze_df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- _source_file: string (nullable = true)



In [7]:
bronze_df.count()

10906858

In [8]:
len(bronze_df.columns)

20

In [9]:
bronze_df.show()

+--------+--------------------+---------------------+---------------+-------------+-------------------+------------------+----------+------------------+-------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|   pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag|  dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|        _source_file|
+--------+--------------------+---------------------+---------------+-------------+-------------------+------------------+----------+------------------+-------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       2| 2016-01-01 00:00:00|  2016-01-01 00:00:00|              

In [13]:
from pyspark.sql import functions as F
unique_counts = bronze_df.select([
    F.countDistinct(F.col(c)).alias(c)
    for c in bronze_df.columns
])

unique_counts.show(truncate=False)

+--------+--------------------+---------------------+---------------+-------------+----------------+---------------+----------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|RatecodeID|store_and_fwd_flag|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|_source_file|
+--------+--------------------+---------------------+---------------+-------------+----------------+---------------+----------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+
|2       |2368616             |2372528              |10             |4513         |35075           |62184          |7    

In [20]:
bob = (bronze_df.groupBy("RatecodeID", "payment_type")\
    .agg(F.sum("total_amount").alias("total_amount")).orderBy("RatecodeID", "payment_type"))

bob.show()

+----------+------------+--------------------+
|RatecodeID|payment_type|        total_amount|
+----------+------------+--------------------+
|         1|           1|1.0882706940980966E8|
|         1|           2| 4.270758721010009E7|
|         1|           3|   360489.3199999912|
|         1|           4|  139397.66000000035|
|         1|           5|                11.8|
|         2|           1|1.0823403620000282E7|
|         2|           2|   3555655.619999874|
|         2|           3|    79994.2800000001|
|         2|           4|  15926.300000000003|
|         3|           1|  1095371.0900000017|
|         3|           2|  339944.85999999964|
|         3|           3|  15492.619999999992|
|         3|           4|  3678.2200000000003|
|         4|           1|  243313.28000000006|
|         4|           2|  104720.75000000012|
|         4|           3|  3534.6600000000003|
|         4|           4|  1205.9499999999996|
|         5|           1|  1915535.5400000138|
|         5| 

In [18]:
bob2 = bronze_df.groupBy("VendorID", "payment_type").agg(\
    F.count("*").alias("n_trips"),
    F.sum("total_amount").alias("total_amount_paid_by_type"),
    F.mean("total_amount").alias("avg_amount_per_type")
).orderBy("VendorID", "payment_type")

bob2.show()

+--------+------------+-------------------+-------------------------+-------------------+
|VendorID|payment_type|count(payment_type)|total_amount_paid_by_type|avg_amount_per_type|
+--------+------------+-------------------+-------------------------+-------------------+
|       1|           1|            3332893|     5.6284750259991676E7| 16.887655937346825|
|       1|           4|              11815|       185350.59000000014| 15.687735082522229|
|       1|           2|            1690599|      2.144391851003002E7|  12.68421341195045|
|       1|           3|              36121|       518482.15999999247| 14.354036709946914|
|       1|           5|                  1|                     11.8|               11.8|
|       2|           1|            3848583|      6.662562913994617E7|  17.31172983405741|
|       2|           4|               1596|      -17150.969999999983|-10.746221804511267|
|       2|           3|               2198|       -25088.40999999996|-11.414199272065495|
|       2|

In [21]:
bronze_df.select(
    F.sum(F.col("VendorID").isNull().cast("int")).alias("null_vendor"),
    F.sum(F.col("payment_type").isNull().cast("int")).alias("null_payment_type"),
    F.sum(F.col("RatecodeID").isNull().cast("int")).alias("null_ratecode"),
    F.sum(F.col("total_amount").isNull().cast("int")).alias("null_total_amount"),
).show()


+-----------+-----------------+-------------+-----------------+
|null_vendor|null_payment_type|null_ratecode|null_total_amount|
+-----------+-----------------+-------------+-----------------+
|          0|                0|            0|                0|
+-----------+-----------------+-------------+-----------------+



In [22]:
bronze_df.groupBy("payment_type").count().orderBy(F.desc("count")).show()
bronze_df.groupBy("RatecodeID").count().orderBy(F.desc("count")).show()


+------------+-------+
|payment_type|  count|
+------------+-------+
|           1|7181476|
|           2|3673651|
|           3|  38319|
|           4|  13411|
|           5|      1|
+------------+-------+

+----------+--------+
|RatecodeID|   count|
+----------+--------+
|         1|10626315|
|         2|  225019|
|         5|   33688|
|         3|   16822|
|         4|    4696|
|        99|     216|
|         6|     102|
+----------+--------+

